In [ ]:
### pre requisites ###

### install spacy
!pip install spacy==2.2.4

### install en model
!python -m spacy download en_core_web_sm

### install stanza
!pip install spacy-stanza==0.2.4

### install stanza model
import stanza
stanza.download("en")

### install truecase
!pip install truecase

### install textstat
!pip install textstat

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 235kB 7.4MB/s 


2021-05-22 16:25:14 INFO: Downloading default packages for language: en (English)...
2021-05-22 16:26:37 INFO: Finished downloading models and saved to /root/stanza_resources.


     |████████████████████████████████| 28.4MB 81kB/s 
     |████████████████████████████████| 102kB 6.3MB/s 
     |████████████████████████████████| 1.9MB 10.9MB/s 


In [ ]:
### install pattern

!sudo apt-get install python-dev default-libmysqlclient-dev
!sudo apt-get install python3-dev
!pip install pattern

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     |████████████████████████████████| 22.3MB 137kB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████

In [ ]:
import spacy
from spacy import displacy
import stanza
from spacy_stanza import StanzaLanguage
snlp = stanza.Pipeline(lang="en")
from pattern.en import comparative, superlative, conjugate, INFINITIVE, PRESENT, PAST, FUTURE, pluralize
import truecase
import re
import textstat

2021-05-22 16:27:10 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-05-22 16:27:10 INFO: Use device: gpu
2021-05-22 16:27:10 INFO: Loading: tokenize
2021-05-22 16:27:21 INFO: Loading: pos
2021-05-22 16:27:22 INFO: Loading: lemma
2021-05-22 16:27:22 INFO: Loading: depparse
2021-05-22 16:27:23 INFO: Loading: sentiment
2021-05-22 16:27:24 INFO: Loading: ner
2021-05-22 16:27:24 INFO: Done loading processors!


In [ ]:
def inorder_traversal(token, removed=None):
  if(len(list(token.children)) == 0):
     return token.text
  s = ""
  if(len(list(token.lefts)) != 0):
    for child in token.lefts:
      if(child != removed):
        s = s + " " + inorder_traversal(child, removed)

  s = s + " " + token.text

  if(len(list(token.rights)) != 0):
    for child in token.rights:
      if(child != removed):
        s = s + " " + inorder_traversal(child, removed)
  return s

def inorder_traversal_2(token, removed):
  if(len(list(token.children)) == 0):
     return token.text
  s = ""
  if(len(list(token.lefts)) != 0):
    for child in token.lefts:
      if(child not in removed):
        s = s + " " + inorder_traversal_2(child, removed)
    
  s = s + " " + token.text

  if(len(list(token.rights)) != 0):
    for child in token.rights:
      if(child not in removed):
        s = s + " " + inorder_traversal_2(child, removed)
  return s

def inorder_traversal_3(token, removed):
  if(len(list(token.children)) == 0):
     return token.text
  s = ""
  if(len(list(token.lefts)) != 0):
    for child in token.lefts:
      if(child not in removed):
        s = s + " " + inorder_traversal_3(child, removed)
  if(token not in removed):
    s = s + " " + token.text

  if(len(list(token.rights)) != 0):
    for child in token.rights:
      if(child not in removed):
        s = s + " " + inorder_traversal_3(child, removed)
  return s

def left_subtree(token):
  if(len(list(token.lefts)) == 0):
     return token.text
  s = ""
  if(len(list(token.lefts)) != 0):
    for child in token.lefts:
      s = s + " " + inorder_traversal(child)
  s = s + " " + token.text
  return s

def left_subtree_2(token, removed=None):
  if(len(list(token.children)) == 0):
     return token.text
  s = ""
  if(len(list(token.lefts)) != 0):
    for child in token.lefts:
      if(child != removed):
        s = s + " " + inorder_traversal(child, removed)

  s = s + " " + token.text
  return s

def left_subtree_3(token, removed):
  if(len(list(token.children)) == 0):
     return token.text
  s = ""
  if(len(list(token.lefts)) != 0):
    for child in token.lefts:
      if(child not in removed):
        s = s + " " + inorder_traversal_3(child, removed)

  if(token not in removed):
    s = s + " " + token.text
  return s


def right_subtree_3(token, removed):
  if(len(list(token.children)) == 0):
     return token.text
  s = ""
  if(len(list(token.rights)) != 0):
    for child in token.rights:
      if(child not in removed):
        s = s + " " + inorder_traversal_3(child, removed)

  if(token not in removed):
    s = s + " " + token.text
  return s

def get_root(doc):
  root  = None
  for token in doc:
    if(token.dep_ == "ROOT"):
      root = token
      return root

def get_root_stanza(doc):
  root  = None
  for token in doc:
    if(token.dep_ == "root"):
      root = token
      return root

def get_tag(token, tag):
  for child in token.children:
    if(child.dep_ == tag):
      return child

def find_in_subtree(token, tag):
  if(token == None):
     return
  if(token.tag_ == tag):
     return token
  for child in token.subtree:
    if(child.dep_ == tag):
      return child

def find_in_children(token, tag):
  if(token == None):
     return
  if(token.tag_ == tag):
     return token
  for child in token.children:
    if(child.dep_ == tag):
      return child

def pos_in_subtree(token, tag):
  if(token == None):
     return False
  if(token.pos_ == tag):
     return True
  for child in token.subtree:
    if(child.pos_ == tag):
      return True
  return False

def auxiliary_verb(verb, subj):
  if subj.tag_ in ("NN", "NNP"):
    if verb.tag_ in ("VBP", "VBZ", "VB"):
      aux = "is "
    elif verb.tag_ in ("VBD", "VBG", "VBN"):
      aux = "was "
    else: 
      aux = "are "
  elif subj.tag_ in ("NNS", "NNPS"):
    if verb.tag_ in ("VBP", "VBZ", "VB"):
      aux = "are "
    elif verb.tag_ in ("VBD", "VBG", "VBN"):
      aux = "were "
    else:
      aux = "are "
  elif subj.tag_ in ("PRP") and subj.text.lower() == "they":
    if verb.tag_ in ("VBP", "VBZ", "VB") :
      aux = "are "
    elif verb.tag_ in ("VBD", "VBG", "VBN"):
      aux = "were "
  elif subj.tag_ in ("PRP"):
    if verb.tag_ in ("VBP", "VBZ", "VB"):
      aux = "is "
    elif verb.tag_ in ("VBD", "VBG", "VBN"):
      aux = "was "
  else:
      aux = "are "
  return aux

def remove_punct(s):
    s_list = list(s.split(" "))
    if(s_list[0] in ['?', '!', ',', ';', '.']):
        s_list[0] = ""
    for i in range(len(s_list)):
        if(len(s_list[i]) > 0 and (s_list[i][0] == "." or s_list[i][0] == ",")):
            s_list[i] = "."
    s = " ".join(c for c in s_list)
    fix_spaces = re.compile(r'\s*([?!.,]+(?:\s+[?!.,]+)*)\s*')
    s = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), s)
    s = truecase.get_true_case(s)
    return s


In [ ]:
def appositive_simplification_2(comp_sent):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(comp_sent)

  root = get_root(doc)
  subj = get_tag(root, "nsubj")

  apposcl = None

  apposcl = find_in_subtree(root, "appos")

  if(apposcl == None):
    apposcl = find_in_subtree(root, "amod")
    if(apposcl != None):
      if(apposcl.head.dep_ != "nsubj"):
        print("Appos: No simplification(no subject)")
        return False, comp_sent
    else:
      print("Appos: No simplification")
      return False, comp_sent

  appos_subj = apposcl.head
  
  appos_phrase = inorder_traversal(apposcl)
  aux_verb = str(auxiliary_verb(root, appos_subj))
  noun_phrase = left_subtree_2(appos_subj, apposcl)
  sentence1 = inorder_traversal(root, apposcl)
  sentence1 = ' '.join(sentence1.split(",")) 
  sentence1 = ' '.join(sentence1.split()) + "."

  sentence2 = noun_phrase + " " + aux_verb + appos_phrase
  sentence2 = ' '.join(sentence2.split(",")) 
  sentence2 = ' '.join(sentence2.split()) + "."

  sentence1 = remove_punct(sentence1)
  sentence2 = remove_punct(sentence2)

  print(sentence1)
  print(sentence2)
  return True, sentence1 + " " + sentence2

appositive_simplification_2("Siddharth, 21 years old, will join the company as an executive")

Siddharth will join the company as an executive.
Siddharth is 21 years old.


(True,
 'Siddharth will join the company as an executive. Siddharth is 21 years old.')

In [ ]:
def rel_order(token):
  if(token.dep_ == "nsubj" or token.dep_ == "nsubjpass"):
    return True
  else:
    return False

def relative_clause_simplify(cmp_snt):
  nlp = spacy.load("en_core_web_sm") 
  doc = nlp(cmp_snt)
  root = get_root(doc)
  relcl = None
  relcl = find_in_subtree(root, "relcl")
  if(relcl == None):
    relcl = find_in_subtree(root, "rcmod")

  if(relcl == None):
    print("Relcl: No simplification")
    return False, cmp_snt

  subj = relcl.head

  rel_phrase = doc[relcl.left_edge.i : relcl.right_edge.i+1].text
  noun_phrase = left_subtree_2(subj, relcl)
  sentence1 = inorder_traversal(root, relcl)
  sentence1 = ' '.join(sentence1.split(",")) 
  sentence1 = ' '.join(sentence1.split()) + "."
  sentence2 = noun_phrase + " " + rel_phrase
  sentence2 = ' '.join(sentence2.split(",")) 
  sentence2 = ' '.join(sentence2.split()) 
  sentence2 = ' '.join(sentence2.split()) + "."
    
  sent1_doc = nlp(sentence1)
  sent1_root = get_root(sent1_doc)
  if(sent1_root.pos_ != "VERB" and sent1_root.pos_ != "AUX"):
    print("Relcl: No simplification")
    return False, cmp_snt

  order = rel_order(subj)
    
  sentence1 = remove_punct(sentence1)
  sentence2 = remove_punct(sentence2)

  if(not order):
    print(sentence1)
    print(sentence2)
    return True, sentence1 + " " + sentence2
  else:
    print(sentence2)
    print(sentence1)
    return True, sentence2 + " " + sentence1

relative_clause_simplify("Prof. Mohan,who teaches art at Delhi University, is a very nice man.")

Prof. Mohan who teaches art at Delhi University.
Prof. Mohan is a very nice man.


(True,
 'Prof. Mohan who teaches art at Delhi University. Prof. Mohan is a very nice man.')

In [ ]:
#nlp = StanzaLanguage(snlp)
snlp = stanza.Pipeline(lang="en")
def conjoint_clause_simplification(cmp_snt,snlp):
  cmp_snt = cmp_snt.rstrip(" ,.")
  #snlp = stanza.Pipeline(lang="en")
  Qnlp = StanzaLanguage(snlp)
  print('the input is ',cmp_snt)
  doc = Qnlp(cmp_snt)
  root = get_root_stanza(doc)

  clause_type = None
    
    
  clause_type = find_in_children(root, "conj")
  if(clause_type == None):
    clause_type = find_in_children(root, "advcl")
  if(clause_type == None):
    clause_type = find_in_children(root, "parataxis")
  if(clause_type == None):
      clause_type = find_in_subtree(root, "conj")
  if(clause_type == None):
    clause_type = find_in_subtree(root, "advcl")
  if(clause_type == None):
    clause_type = find_in_subtree(root, "parataxis")
  if(clause_type == None):
    print("Conjoint: No simplification(no second clause found)")
    return False, cmp_snt

  clause_root = None

  if(clause_type.dep_ == "conj"):
    clause_root = find_in_subtree(root, "cc")
    if(clause_root != None):
      if((not pos_in_subtree(root, "VERB")) or (not (pos_in_subtree(clause_type, "VERB") or pos_in_subtree(clause_type, "AUX")))):
        print("Conjoint: No simplification 2(no verb in second clause)")
        return False, cmp_snt
    else:
      clause_root = find_in_children(clause_type, "advmod")
  elif(clause_type.dep_ == "advcl"):
    print("advcl")
    clause_root = find_in_children(clause_type, "mark")
    if(clause_root == None):
      clause_root = find_in_subtree(root, "mark")
    if(clause_root == None):
      clause_root = find_in_children(clause_type, "advmod")
    if(clause_root != None):
        if(clause_root.text == "to"):
            print("Conjoint: No simplification 3(no conjunction found)")
            return False, cmp_snt
  elif(clause_type.dep_ == "parataxis"):
    print("parataxis")
    clause_root = find_in_children(clause_type, "advmod")
  else:
    print("Conjoint: No simplification 3(no conjunction found)")
    return False, cmp_snt
  

  if(clause_root == None):
    print("Conjoint: No simplification 4")
    return False, cmp_snt
  
  clause_subj = None

  clause_subj = find_in_children(clause_type, "nsubj")
  if(clause_subj == None):
    clause_subj = find_in_children(clause_type, "nsubj:pass")
  clause_subj_flag = False
  if(clause_subj == None):
    clause_subj_flag = True
    clause_subj = find_in_children(root, "nsubj")
    if(clause_subj == None):
        clause_subj = find_in_children(root, "nsubj:pass")
    if(clause_subj == None):
        print("Conjoint: No simplification(no clause subject found)")
        return False, cmp_snt
    aux_str = ""
    for child in root.children:
        if(child.dep_ == "aux"):
            aux_str += child.text + " "
  modal = None
  if(clause_root.text.lower() in ['when', 'after', 'since', 'before', 'once']):
    modal = find_in_subtree(root, "aux")

  marker1 = ""
  marker2 = ""
  if(clause_root.text.lower() == "if"):
    marker1 = "Then"
    for child in root.subtree:
      if(child.text.lower() == "then"):
        marker1 = ""

  sentence1 = inorder_traversal_2(root, [clause_type, clause_root])
  
  sentence2 = ""
  marker2 = add_marker(clause_type, clause_root, modal) + " "
  if(clause_subj_flag):
    sentence2 = clause_subj.text + " " + aux_str + inorder_traversal(clause_type, clause_root)
  else:
    sentence2 = inorder_traversal(clause_type, clause_root)

  reverse = None

  if((root.i > clause_type.i) or (clause_root.text.lower() in ["because", "as"] and clause_root.i > 0)):
    if(clause_root.text.lower() in ['when', 'after', 'since', 'before', 'once'] and clause_root.i == 0):
      reverse = False
    else:
      reverse = True
  else:
    reverse = False
    
    
  if(clause_root.text.lower() in ["because", "as"] and clause_root.i > 0):
    sentence1 = marker2 + sentence1
    sentence2 = marker1 + sentence2
  else:
    sentence1 = marker1 + sentence1
    sentence2 = marker2 + sentence2
    
  s1 = remove_punct(sentence1)
  s2 = remove_punct(sentence2)
  exception_list = ["This happened", "This happens", "This was", "This is"]
  for c in ['when', 'after', 'since', 'before', 'once']:
        exception_list.append("This is " + c)
        exception_list.append("This was " + c)
        exception_list.append("This happens " + c)
        exception_list.append("This happened " + c)
        if(modal != None):
             exception_list.append('This ' + modal.text + ' happen ' + c)
  if(s1 in exception_list or s2 in exception_list):
        return False, cmp_snt
    
  if(reverse):
    print(sentence2)
    print(sentence1)
    return True, sentence2 + ". " + sentence1 + "."
  else:
    print(sentence1)
    print(sentence2)
    return True, sentence1 + ". " + sentence2 + "."

def add_marker(root, conj,  modal=None):
  conj_text = conj.text.lower()
  if(conj_text in ["and", "moreover"]):
    return "And"
  elif(conj_text == "if"):
    return "Suppose"
  elif(conj_text in ['though', 'although', 'but', 'whereas', 'however']):
    return "But"
  elif(conj_text in ['when', 'after', 'since', 'before', 'once']):
    if(conj.i > 0):
      if root.tag_ == 'VBP' or root.tag_ == 'VBZ' or root.tag_ == 'VB':
        return 'This is ' + conj_text + " "
      else:
        return 'This was ' + conj_text + " "
    else:
      if root.tag_ == 'VBP' or root.tag_ == 'VBZ':
        return 'This happens ' + conj_text + " "
      elif root.tag_ == 'VB' and modal != None: 
        return 'This ' + modal.text + ' happen ' + conj_text + " "
      else:
        return 'This happened ' + conj_text + " "
  elif conj_text in ['because', 'so', 'while', "therefore"]:
    return 'So '
  elif conj_text in ["nevertheless", "nonetheless", "yet"]:
    return 'But'
  elif conj_text == "otherwise":
    return 'or'
  elif conj_text == "unless":
    return 'unless'
  else:
     return ""
a,b = conjoint_clause_simplification("Ram was determined to get high marks nevertheless he did not pass.",snlp)
remove_punct(b)

2021-05-22 16:27:33 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-05-22 16:27:33 INFO: Use device: gpu
2021-05-22 16:27:33 INFO: Loading: tokenize
2021-05-22 16:27:33 INFO: Loading: pos
2021-05-22 16:27:34 INFO: Loading: lemma
2021-05-22 16:27:34 INFO: Loading: depparse
2021-05-22 16:27:35 INFO: Loading: sentiment
2021-05-22 16:27:36 INFO: Loading: ner
2021-05-22 16:27:36 INFO: Done loading processors!


the input is  Ram was determined to get high marks nevertheless he did not pass
parataxis
 Ram was determined  to get  high marks
But  he did not pass


'Ram was determined to get high marks. But he did not pass.'

In [ ]:
a,b = conjoint_clause_simplification(" bzö  is in favor  of a referendum  about the lisbon treaty  but against an eu-withdrawal.",snlp)
remove_punct(b)

the input is   bzö  is in favor  of a referendum  about the lisbon treaty  but against an eu-withdrawal
Conjoint: No simplification 2(no verb in second clause)


'Bzö is in favor of a referendum about the Lisbon treaty but against an Eu-Withdrawal'

In [ ]:
def change_verb_form(verb, v_aux, auxpass, subj_tag, subj_word):
    word_list = v_aux.split()
    new_verb = ""
    modal_list = ["can", "could", "must", "should", "will", "would", "may", "might", "shall"]
    if ('will' in word_list or 'would' in word_list) and auxpass != "been":
      print("future simple")
      new_verb = v_aux + conjugate(verb, 'pl')
    elif auxpass == 'been':
      if 'will' in word_list or 'would' in word_list:
        print("future perfect")
        new_verb = v_aux + verb
      elif 'has' in word_list or 'have' in word_list:
        print("present perfect")
        if bool(set(word_list) & set(modal_list)):
            new_verb = v_aux + verb
        elif subj_tag in ("NNS", "NNPS") :
            new_verb = "have " + verb
        elif subj_tag in ("NN", "NNP"):
            new_verb = "has " + verb
        elif subj_tag in ("PRP") and subj_word.lower() in ("them", "me", "you"):
            new_verb = "have " + verb
        elif subj_tag in ("PRP") and subj_word.lower() in ("it", "her", "him"):
            new_verb = "has " + verb
        else:
            new_verb = v_aux + verb
      elif 'had' in word_list:
        print("past perfect")
        new_verb = v_aux + verb
    elif auxpass == "being":
      if('was' in word_list or 'were' in word_list):
        print("past continous")
        if subj_tag in ("NNS", "NNPS"):
            new_verb = "were " + conjugate(verb, tense='present', aspect='progressive') 
        elif subj_tag in ("NN", "NNP"):
            new_verb = "was " + conjugate(verb, tense='present', aspect='progressive') 
        elif subj_tag in ("PRP") and subj_word.lower() in ("them", "you"):
            new_verb = "were " + conjugate(verb, tense='present', aspect='progressive')
        elif subj_tag in ("PRP") and subj_word.lower() in ("me", "it", "her", "him"):
            new_verb = "was " + conjugate(verb, tense='present', aspect='progressive')
        else:
            new_verb = "was " + conjugate(verb, tense='present', aspect='progressive')
      else:
        print("present continous")
        if subj_tag in ("NNS", "NNPS"):
            new_verb = "are " + conjugate(verb, tense='present', aspect='progressive') 
        elif subj_tag in ("NN", "NNP"):
            new_verb = "is " + conjugate(verb, tense='present', aspect='progressive') 
        elif subj_tag in ("PRP") and subj_word.lower() in ("them", "you"):
            new_verb = "are " + conjugate(verb, tense='present', aspect='progressive')
        elif subj_tag in ("PRP") and subj_word.lower() in ("it", "her", "him"):
            new_verb = "is " + conjugate(verb, tense='present', aspect='progressive')
        elif subj_tag in ("PRP") and subj_word.lower() in ("me"):
            new_verb = "am " + conjugate(verb, tense='present', aspect='progressive')
        else:
            new_verb = "is " + conjugate(verb, tense='present', aspect='progressive')
    elif auxpass in ['was', 'were']:
      print("simple past")
      if subj_tag in ("NNS", "NNPS"):
        new_verb = conjugate(verb, 'ppl') + " "
      elif subj_tag in ("NN", "NNP"):
        new_verb = conjugate(verb, '3sgp') + " "
      elif subj_tag in ("PRP") and subj_word.lower() in ("them", "me", "you"):
        new_verb = conjugate(verb, 'ppl') + " "
      elif subj_tag in ("PRP") and subj_word.lower() in ("it", "her", "him"):
        new_verb = conjugate(verb, '3sgp') + " "
      else:
        new_verb = conjugate(verb, '3sgp') + " "
    else:
      print("simple present")
      if subj_tag in ("NNS", "NNPS"):
        new_verb = conjugate(verb, 'pl') + " "
      elif subj_tag in ("NN", "NNP"):
        new_verb = conjugate(verb, '3sg') + " "
      elif subj_tag in ("PRP") and subj_word.lower() in ("them", "me", "you"):
        new_verb = conjugate(verb, 'pl') + " "
      elif subj_tag in ("PRP") and subj_word.lower() in ("it", "her", "him"):
        new_verb = conjugate(verb, '3sg') + " "
      else:
        new_verb = conjugate(verb, '3sg') + " "

    return new_verb

def change_pronoun_object(word):
  pron_list = {'me':'I', 'you':'you', 'him':'he', 'her':'she', 'it':'it', 'them':'they', 'us':'we'}
  word_list = word.split()
  for i in range(len(word_list)):
    if(word_list[i] in pron_list.keys()):
        word_list[i] = pron_list[word_list[i]]
  return " ".join(str(e) for e in word_list)

def change_pronoun_subject(word):
  pron_list = {'i':'me', 'you':'you', 'he': 'him', 'she':'her', 'it':'it', 'they':'them', 'we':'us'}
  word_list = word.split()
  word_list = word.split()
  for i in range(len(word_list)):
    if(word_list[i].lower() in pron_list.keys()):
        word_list[i] = pron_list[word_list[i].lower()]
  return " ".join(str(e) for e in word_list)



In [ ]:
def passive_simplify(cmp_snt):
    nlp=spacy.load('en_core_web_sm')
    doc = nlp(cmp_snt)
    simplify_flag = False
    root = get_root(doc)
    auxpass = find_in_children(root, "auxpass")
    if(auxpass == None):
        print("No passive clause found")
        return simplify_flag, cmp_snt
    agent = find_in_children(root, "agent")
    if(agent == None):
        print("No agent found")
        return simplify_flag, cmp_snt
    subj = find_in_children(root, "nsubjpass")
    if(subj == None):
        print("No subject found")
        return simplify_flag, cmp_snt
    
    simplify_flag = True
    
    aux = find_in_children(root, "aux")
    
    aux_tense = auxpass.tag_
    if aux_tense == 'VB' and aux != None:
        aux_tense = aux.tag_
    elif aux_tense == 'VBN' and aux != None:
        if aux.text.lower() not in ("has", "have", "had"):
            aux_tense = 'MD'

    new_obj = inorder_traversal(subj)
    obj = find_in_children(agent, "pobj")
    new_subj = inorder_traversal(obj)
    removed_aux = []
    for child in root.children:
        if(child.dep_ == "aux"):
            removed_aux.append(child)
    
    left_of_subj = ""
    right_of_subj = ""
    left_of_obj = ""
    right_of_obj = ""
    
    flag = 0
    for child in root.lefts:
        if(child.dep_ == "nsubjpass"):
            flag = 1
            continue
        if(flag == 0 and child not in [root, subj, auxpass, agent] + removed_aux):
            left_of_subj += inorder_traversal(child) + " "
        elif(flag == 1 and child not in [root, subj, auxpass, agent] + removed_aux):
            right_of_subj += inorder_traversal(child) + " "
            
    flag = 0
    for child in root.rights:
        if(child.dep_ == "agent"):
            continue
        if(((child.dep_ == "prt" or child.dep_ == "prep") and (len(list(child.children)) == 0)) and child not in [root, subj, auxpass, agent] + removed_aux):
            left_of_obj += inorder_traversal(child) + " "
        elif(child not in [root, subj, auxpass, agent] + removed_aux): 
            right_of_obj += inorder_traversal(child) + " "
    
    v_aux = ""
    if(aux != None):
        v_aux = aux.text
    aux_str = ""
    for child in root.children:
        if(child.dep_ == "aux"):
            aux_str += child.text + " "
    new_verb = change_verb_form(root.text, aux_str, auxpass.text, obj.tag_, obj.text)
    new_subj = change_pronoun_object(new_subj)
    new_obj = change_pronoun_subject(new_obj)
    final_sent = left_of_subj + " " + new_subj + " " + right_of_subj + " " + new_verb + " " + left_of_obj + " " + new_obj + " " + right_of_obj
    final_sent = ' '.join(final_sent.split())   
    final_sent = truecase.get_true_case(final_sent)
    final_sent = remove_punct(final_sent)
    print(final_sent)
    return simplify_flag, final_sent


In [ ]:
def simplify(snt):
    Qnlp = StanzaLanguage(snlp)
    doc=Qnlp(snt)
    flag = False
    final_snt = ""
    for s in doc.sents:
        flag, simp_snt = appositive_simplification_2(s.text)
        if(flag == True):
            print(simp_snt)
            final_snt += (simplify(simp_snt) + " ")
            continue
        else:
            flag, simp_snt = conjoint_clause_simplification(s.text,snlp)
            if(flag == True):
                print(simp_snt)
                final_snt += (simplify(simp_snt) + " ")
                continue
            else:
                flag, simp_snt = relative_clause_simplify(s.text)
                if(flag == True):
                    final_snt += (simplify(simp_snt) + " ")
                    continue
                else:
                    flag, simp_snt = passive_simplify(s.text)
                    if(flag == True):
                        final_snt += (simplify(simp_snt) + " ")
                        continue
                    else:
                        final_snt += (simp_snt + " ")
    return final_snt

In [ ]:
sentence = "John , who was infected with Covid-19, had quit, before his band released a studio album."
sentence_simplified = simplify(sentence)
sentence_simplified = remove_punct(sentence_simplified)

Appos: No simplification
the input is  John , who was infected with Covid-19, had quit, before his band released a studio album
advcl
  John ,  who was infected  with Covid - 19 , had quit ,
This was before    his band released  a studio album
  John ,  who was infected  with Covid - 19 , had quit ,. This was before    his band released  a studio album.
Appos: No simplification
the input is    John ,  who was infected  with Covid - 19 , had quit
Conjoint: No simplification(no second clause found)
John who was infected with Covid- 19.
John had quit.
Appos: No simplification
the input is  John who was infected with Covid-
Conjoint: No simplification(no second clause found)
Relcl: No simplification
No passive clause found
Appos: No simplification
the input is  19
Conjoint: No simplification(no second clause found)
Relcl: No simplification
No passive clause found
Appos: No simplification
the input is  John had quit
Conjoint: No simplification(no second clause found)
Relcl: No simplificatio

In [ ]:
print(sentence_simplified)
orig_score = textstat.flesch_reading_ease(sentence)
final_score = textstat.flesch_reading_ease(sentence_simplified)
print(orig_score)
print(final_score)

John who was infected with Covid- 19. John had quit. This was before his band released a studio album.
64.71
90.46


In [ ]:
sentence = "Bob, 31 years old, now qualifies the criteria for nonexecutive director"
sentence_simplified = simplify(sentence)
#sentence_simplified = remove_punct(sentence_simplified)

Bob now qualifies the criteria for nonexecutive director.
Bob is 31 years old.
Bob now qualifies the criteria for nonexecutive director. Bob is 31 years old.
Appos: No simplification(no subject)
the input is  Bob now qualifies the criteria for nonexecutive director
Conjoint: No simplification(no second clause found)
Relcl: No simplification
No passive clause found
Appos: No simplification
the input is  Bob is 31 years old
Conjoint: No simplification(no second clause found)
Relcl: No simplification
No passive clause found


In [ ]:
print(sentence_simplified)
orig_score = textstat.flesch_reading_ease(sentence)
final_score = textstat.flesch_reading_ease(sentence_simplified)
print(orig_score)
print(final_score)

Bob now qualifies the criteria for nonexecutive director. Bob is 31 years old.  
43.39
47.96


In [ ]:
flag, s = appositive_simplification_2("Bob, 31 years old, now qualifies the criteria for nonexecutive director")

Bob now qualifies the criteria for nonexecutive director.
Bob is 31 years old.


In [ ]:
flag, s = relative_clause_simplify("Prof Mohan, who teaches art at Delhi University, is a very nice man.")

Prof Mohan who teaches art at Delhi University.
Prof Mohan is a very nice man.


In [ ]:
flag, s = conjoint_clause_simplification("She likes to go the tennis club because she likes to play tennis",snlp)
remove_punct(s)

the input is  She likes to go the tennis club because she likes to play tennis
advcl
 she likes  to play tennis
So   She likes  to go  the tennis club


'She likes to play tennis. So she likes to go the tennis club.'

In [ ]:
flag, s = conjoint_clause_simplification("Ram was determined to get high marks nevertheless he did not pass",snlp)
remove_punct(s)

the input is  Ram was determined to get high marks nevertheless he did not pass
parataxis
 Ram was determined  to get  high marks
But  he did not pass


'Ram was determined to get high marks. But he did not pass.'

In [ ]:
flag, s = passive_simplify("His training regimen had been kept up nicely for a month by him")